In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.client.ml import MlClient
import pandas as pd

es = Elasticsearch(['http://localhost:9200/'])

In [2]:
# .ml-anomalies-fq-mean-split-1403/_search?size=2000
from cytoolz.dicttoolz import merge

res = es.search(index=".ml-anomalies-fq-mean-split-1403", body={"size": 1500, "query": {"query_string": { "query": "result_type:influencer"}}})
data = map(lambda s: s['_source'], res['hits']['hits'])
data = list(data)

influencers = pd.DataFrame.from_dict(data)
influencers = influencers[['timestamp', 'influencer_score', 'airline']]
influencers

,timestamp,influencer_score,airline
0,1549513800000,12.736863,AMX
1,1549514700000,48.668256,AWE
2,1549514700000,14.604029,FFT
3,1549535400000,14.631551,ASA
4,1549546200000,33.545081,SWA
...,...,...,...
193,1549551600000,10.753930,DAL
194,1549729800000,17.202460,AAL
195,1549714500000,14.086020,SWR
196,1549819800000,26.747120,UAL


In [3]:
ml_client = MlClient(es)

job_id = 'fq-mean-split-1403'

response = ml_client.get_overall_buckets(job_id)

In [4]:
df = pd.DataFrame.from_dict(response['overall_buckets'])

In [5]:
import altair as alt

alt.Chart(df, title='Overall Buckets for {}'.format(job_id)).mark_rect(stroke='lightgray').encode(
    x=alt.X('hours(timestamp):T', title=' '),
    color=alt.Color(
        'max(overall_score):Q',
        scale=alt.Scale(
            domain=[0,.001,.001,25, 25, 50, 50, 75, 75, 100],
            range=['white', 'white', 'lightblue', 'lightblue', 'yellow', 'yellow', 'orange', 'orange', 'red', 'red']
    )),
    tooltip=[
        alt.Tooltip('hours(timestamp):T', title='Date'),
        alt.Tooltip('max(overall_score):Q', title='Score')
    ]
).properties(width=750).configure_axis(
    domainColor='white', tickColor="lightgray"
).configure_view(
    strokeOpacity=0
)

alt.Chart(...)

In [10]:
import altair as alt

alt.Chart(
    influencers,
    title='Overall Buckets for {}'.format(job_id)
).mark_rect().encode(
    x=alt.X('hours(timestamp):T', title=' ', axis=alt.Axis(grid=True, tickCount=30)),
    y=alt.Y('airline:O', axis=alt.Axis(bandPosition=1, grid=True)),
    color=alt.Color(
        'max(influencer_score):Q',
        scale=alt.Scale(
            domain=[0,25, 25, 50, 50, 75, 75, 100],
            range=['lightblue', 'lightblue', 'yellow', 'yellow', 'orange', 'orange', 'red', 'red']
    )),
    tooltip=[
        alt.Tooltip('hours(timestamp):T', title='Date'),
        alt.Tooltip('max(influencer_score):Q', title='Score')
    ]
).properties(
    width=750
).configure_axis(
    domainColor='lightgray', tickColor='lightgray'
).configure_view(
    strokeOpacity=0
)

alt.Chart(...)